# Textos versus Bytes

Converter de Unicode para bytes é codificar (usando UTF8 por exemplo)

In [11]:
#encoding -> Unicode para Bytes
a = "Ó man, estou decodado".encode("utf8")
a

b'\xc3\x93 man, estou decodado'

In [13]:
# decode
a = b'abc'
a.decode()

'abc'

### Sequencias binárias

Em Python 3 temos dois tipos de sequencias binárias
```python
bytes #imutável
bytearray #mutável
``` 
Quando caractéres estão dentro do intervalo ASCII, eles são apresentados as-is, mas os que estão fora, são representados por uma sequência escape hexadecimal.

É possível usar diversos métodos de `str` nos `bytes`.

In [58]:
# Caracteres ASCII são apresentados as-is
print(bytes("ë a A", "utf8"))

# Sequencias Binárias tem diversos métodos em comum com str, mas somente com outros bytes

a= bytes('a:á:à:a',"utf_8")
print(a.upper(),
a.endswith(b"a"),
a.replace(b"a", b"b"))

bytes.fromhex("61 6D 6F 6D").decode()

# Criando bytes

str(b"aaaa", encoding="utf_8")

b'\xc3\xab a A'
b'A:\xc3\xa1:\xc3\xa0:A' True b'b:\xc3\xa1:\xc3\xa0:b'


'aaaa'

## Codificadores/Decodificadores

In [62]:
for codec in ["latin_1", "utf8", "utf16"]:
    print(codec, "El Niño".encode(codec), sep="\n")
    

latin_1
b'El Ni\xf1o'
utf8
b'El Ni\xc3\xb1o'
utf16
b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## Entendendo problemas de codificação
`UnicodeEncodeError`

`UnicodeDecodeError`

`SyntaxError`

In [68]:
city = "São Paulo"
print(city.encode("utf8"))
print(city.encode("utf16"))
print(city.encode("latin1"))
print(city.encode("cp437"))


b'S\xc3\xa3o Paulo'
b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
b'S\xe3o Paulo'


UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [75]:
print(city.encode("cp437", errors="ignore").decode())
print(city.encode("cp437", errors="xmlcharrefreplace").decode())
print(city.encode("cp437", errors="replace").decode())

So Paulo
S&#227;o Paulo
S?o Paulo


In [80]:
# Geração de gremlins ou UnicodeDecodeError com codificações de 8bits
octets = b'Montr\xe9al'
print(octets.decode('cp1252'))
print(octets.decode('iso8859_7'))
print(octets.decode('koi8_r'))
print(octets.decode('utf_8'))


Montréal
Montrιal
MontrИal


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

#### **SyntaxError ao carregar módulos com codificação inesperada**
A codificação de fonte padrão é a UTF8. Ao carregar módulos `.py` que não sejam UTF8 sem declaração da codificação, vai dar ruim.

A declaraçao para coding específico é feita no início do arquivo assim:

```python
# coding: cp1252

print("Olá, Mundo")
```

#### **É possível usar identificadores de variáveis que não são ASCII no código fonte**

In [82]:
çáà = "Olá"

### **Como descobrir a codificação????**

Não pode! Somente através de métodos Heurísticos e estatísticos.

Para isso existe a lib `chardet`

In [92]:
u16 = 'El Niño'.encode("utf16")
print(u16) # little-endian CPU Intel, o BOM (byte-mark order) vem primeiro, no exemplo acima o E (representado por 69 e 0) só aparecem nos offsets 2 e 3 
print(list(u16))

u16_little_endian = 'El Niño'.encode("utf_16le")
print(u16_little_endian)
list(u16_little_endian)

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'
[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
b'E\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

### **Lidando com arquivos texto**

Padrão sanduíche:
 
   
 
----- ------> **Decodifica Bytes na entrada**

|-----------| --------> **Processa somente em texto**

 ----------> **Codifica na saída**
 
 O `open` aplica esse padrão!

### **Comparação entre Strings e normalizações**

In [5]:
c = "café"
c1 = "cafe\u0301"
print(c,c1)
print(len(c),len(c1))
c == c1

café café
4 5


False

In [9]:
from unicodedata import normalize

print(normalize("NFC", c) == normalize("NFC", c1))
print(len(normalize("NFC", c)), len(normalize("NFC", c1)))
print(len(normalize("NFD", c)), len(normalize("NFD", c1)))

True
4 4
5 5


In [ ]:
# Case folding

### **Ordenação de texto Unicode**

É preciso lidar com `locale` para resolver problemas de ordenação de caracteres não `ASCII`

In [11]:
frutas = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(frutas)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

In [13]:
import locale
locale.setlocale(locale.LC_COLLATE, "pt_BR.UTF-8")
frutas = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(frutas, key=locale.strxfrm)

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

### `PyUCA` solução mais simples

In [20]:
import pyuca

coll=pyuca.Collator()
frutas = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(frutas, key=coll.sort_key)

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [21]:
import sys
sys.maxunicode

1114111